In [6]:
!pip install \
    transformers[sentencepiece,torch] \
    torch \
    sentence-transformers \
    pinecone-client[grpc] \
    langchain[all] \
    huggingface_hub \
    kaggle \
    bitsandbytes \
    llama-index  \
    python-dotenv


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_KEY = os.getenv("PINECONE_KEY")
HUGGING_TOKEN = os.getenv("HUGGING_TOKEN")
I_NAME = os.getenv("INDEX_NAME")
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

#Carga de documentos

In [ ]:
# Upload Kaggle API key
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d alfathterry/bbc-full-text-document-classification

# Unzip the dataset
!unzip bbc-full-text-document-classification.zip -d /content

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/alfathterry/bbc-full-text-document-classification
License(s): MIT
  0% 0.00/1.84M [00:00<?, ?B/s]
100% 1.84M/1.84M [00:00<00:00, 116MB/s]
Archive:  bbc-full-text-document-classification.zip
  inflating: /content/bbc_data.csv   


In [ ]:
import pandas as pd

bbc_data = pd.read_csv(r"/content/bbc_data.csv")
df = pd.DataFrame(bbc_data, columns=['data', 'labels'])
df.info()
df.sample(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2225 non-null   object
 1   labels  2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


,data,labels
349,Howl helps boost Japans cinemas Japans box of...,entertainment
1678,Blair says mayor should apologise Tony Blair ...,politics
293,Tautou to star in Da Vinci film French actres...,entertainment
2087,Parents face video game lessons Ways of ensur...,tech
1895,Games enter the classroom Video games could s...,tech
1194,2004: An Irish Athletics Year 2004 wont be re...,sport
387,Aids and climate top Davos agenda Climate cha...,business
854,Japans ageing workforce: built to last In his...,business
956,Rusedski forced out in Marseille Greg Rusedsk...,sport
160,Tarantino to direct CSI episode Film director...,entertainment


In [ ]:
df.info()

df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    2225 non-null   object
 1   labels  2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


,0
data,0
labels,0


In [ ]:
df_subset= df[(df['labels'] == 'sport') | (df['labels'] == 'business') | (df['labels'] == 'politics')]
df_subset.sample(10)

,data,labels
1266,Israeli club look to Africa Four African play...,sport
611,Electronics firms eye plasma deal Consumer el...,business
922,Johnson edges out rival Sotherton Jade Johnso...,sport
958,Spain coach faces racism inquiry Spains Footb...,sport
1035,Dominici backs lacklustre France Wing Christo...,sport
1370,Chelsea hold Arsenal A gripping game between ...,sport
444,"Asia shares defy post-quake gloom Indonesian,...",business
1488,EU fraud clampdown urged EU member states are...,politics
513,Optimism remains over UK housing The UK prope...,business
654,Macys owner buys rival for $11bn US retail gi...,business


In [ ]:
import re
import string
import nltk
from nltk.tokenize import sent_tokenize


nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()

    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Eliminar signos de puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Eliminar caracteres especiales (cualquier cosa que no sea letra o número)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Eliminar dobles espacios (convertir múltiples espacios a un solo espacio)
    text = re.sub(r'\s+', ' ', text)

    return text

def segment_text_into_paragraphs(text, max_sentences=4):

    #Dividir texto por espacios
    words = text.split(" ")
    paragraphs = []
    #Hacer conjuntos de 4 palabras e insertar a paragraphs
    for i in range(0, len(words), max_sentences):
        paragraph = ' '.join(words[i:i + max_sentences])
        paragraphs.append(paragraph)

    return paragraphs

In [ ]:
display(df_subset.loc[918])

,918
data,Man City 0-2 Man Utd Manchester United reduce...
labels,sport


In [ ]:
df_subset['cleaned_data'] = df_subset['data'].apply(lambda x: clean_text(x))
df_subset['segmented_data'] = df_subset['cleaned_data'].apply(lambda x: segment_text_into_paragraphs(x))

<ipython-input-12-96474ae49a37>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['cleaned_data'] = df_subset['data'].apply(lambda x: clean_text(x))
<ipython-input-12-96474ae49a37>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['segmented_data'] = df_subset['cleaned_data'].apply(lambda x: segment_text_into_paragraphs(x))


In [ ]:
display(df_subset.loc[918])
df_subset.to_csv('/content/bbc-text-cleaned-segmented.csv', index=False)

,918
data,Man City 0-2 Man Utd Manchester United reduce...
labels,sport
cleaned_data,man city 02 man utd manchester united reduced ...
segmented_data,"[man city 02 man, utd manchester united reduce..."


In [ ]:
from sentence_transformers import SentenceTransformer

data_path = '/content/bbc-text-cleaned-segmented.csv'
df = pd.read_csv(data_path)

model = SentenceTransformer('all-MiniLM-L6-v2')

# Función para generar embeddings
def generate_embeddings(texts):
    return model.encode(texts).tolist()

embeddings = generate_embeddings(df['segmented_data'].explode())

print(f"Total de embeddings generados: {len(embeddings)}")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total de embeddings generados: 1438


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import pinecone

pc = Pinecone(api_key=PINECONE_KEY)

# Crear un índice en Pinecone si aún no existe
index_name = "bbc-text-index"
'''
if index_name not in pc.list_indexes():
    pc.create_index(index_name,
                    dimension=len(embeddings[0]),
                    metric='cosine',
                    spec=ServerlessSpec(
                      cloud='aws',
                      region='us-east-1'
                    ))  # Usamos 'cosine' como métrica
'''
# Conectar al índice creado
index = pc.Index(index_name)

print(f"Conectado al índice: {index_name}")

Conectado al índice: bbc-text-index


In [ ]:
# Preparar los datos para Pinecone
ids = [str(i) for i in range(len(embeddings))]
metadata = [{"text": text} for text in df['segmented_data'].explode()]

batch_size = 100

# Subir los embeddings en batches
for i in range(0, len(embeddings), batch_size):
    batch_embeddings = embeddings[i:i + batch_size]
    batch_ids = ids[i:i + batch_size]
    batch_metadata = metadata[i:i + batch_size]

    index.upsert(vectors=zip(batch_ids, batch_embeddings, batch_metadata))

print(f"Total de embeddings insertados: {len(embeddings)}")

Total de embeddings insertados en Pinecone: 1438


In [ ]:


import importlib
import bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM
import pinecone
import torch



device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available, otherwise CPU

# Initialize Pinecone
pc = pinecone.Pinecone(api_key=PINECONE_KEY)
index = pc.Index(I_NAME)

# Load model and tokenizer, using the correct dtype if the device is CUDA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == 'cuda' else torch.float32,  # Auto-select dtype
    load_in_4bit=True,
    device_map="auto", # Important: Map the model to the appropriate device
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == 'cuda' else torch.float32, # Auto-select dtype
)



ModuleNotFoundError: No module named 'bitsandbytes'

In [ ]:

from sentence_transformers import SentenceTransformer
import torch

# Load the sentence-transformers model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

def query_and_answer(query_text):
    # Embed the query text using sentence-transformers
    xq = sentence_model.encode(query_text).tolist()  # Embed query using sentence-transformers and ensure it is a list

    # Perform the query
    res = index.query(vector=xq, top_k=1, include_metadata=True)
    print(res)
    if not res['matches']:
        return "No se encontraron resultados relevantes para tu pregunta."

    # Extract context and score
    context = res['matches'][0]['metadata']['text']
    score = res['matches'][0]['score']  # Assuming score represents the match precision

    # Define reliability message based on score
    if score >= 0.50:
        reliability_message = "Esta respuesta es fiable."
    elif 0.30 <= score < 0.50:
        reliability_message = "Esta respuesta puede contener errores."
    else:
        reliability_message = "Esta respuesta contiene ciertos errores."

    prompt = f"""
    Context: {context}
    Question: {query_text}
    Answer:
    """

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the answer
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Remove the prompt from the generated text
    answer = answer[len(prompt):]

    # Combine reliability message with the answer
    return f"{reliability_message}\n{answer.strip()}"


In [1]:

question = "show the code of a hello world in C++" #@param {type:"string"}
print(query_and_answer(question))

NameError: name 'query_and_answer' is not defined